# Introduction
This is the code of **QCOS** in real device.

# Readme
1. Please run the following code blocks one by one, and finnally you will get the result in the final block.
2. If you want to change the dataset, please modify the next code block.

In [1]:
lstDatabase=[[1, '/dataset_evaluation_storage_0'], [3, '/dataset_evaluation_storage_1'], [2, '/dataset_evaluation_storage_2']]

#################
# lstDatabase[i] means: Read dataset_0 in following code
# Num_Qubit_Data means: Number of qubits required for storage determined by data type
#################
Num_Qubit_Data, curDb = lstDatabase[0]

print("Finished!")

Finished!


In [6]:
import os
import csv
import numpy as np
import pandas as pd

from qiskit import QuantumCircuit, Aer, assemble, QuantumRegister, ClassicalRegister
from math import pi
import math
import numpy as np
from qiskit.visualization import plot_bloch_multivector, plot_histogram
from qiskit.circuit.library.standard_gates import XGate, HGate, ZGate


from qiskit import QuantumCircuit, Aer, assemble, QuantumRegister, ClassicalRegister
from math import pi
import math
import numpy as np


from qiskit.visualization import plot_bloch_multivector, plot_histogram
from qiskit.circuit.library.standard_gates import XGate, HGate, ZGate
from qiskit import execute
import pandas as pd
from qiskit.circuit.library import MCMT


import pandas as pd

import itertools

def getDatabase(filename):
    pathName = os.getcwd() + filename

    numFiles = []

    fileNames = os.listdir(pathName)
    fileNames.reverse()
    for fileName in fileNames:
        if fileName.endswith(".csv"):
            numFiles.append(fileName)


    Database = {}
    Database['tables'] = []
    table_id = 0
    num_table = 0
    for i in numFiles:
        file = open(os.path.join(pathName, i), "rU")
        reader = csv.reader(file, delimiter=',')

        id = 0
        table_temp = []
        for row in reader:
            if id == 0:
                num_attributes = len(row)
            row_temp = [table_id] + [id] + list(map(int, row))
            table_temp.append(row_temp)
            id = id + 1
        tableHead = ["tableid"] + ["id"] + [str(i) for i in range(num_attributes)]
        indexHead = ["row_" + str(i) for i in range(id)] 
        relation = pd.DataFrame(table_temp, columns = tableHead, index = indexHead)

        Database['tables'].append( {
            "table_id" : table_id,
            "num_attributes" : num_attributes,
            "num_row" : id,
            "relation" : relation,
        })
        table_id = table_id + 1
        num_table = num_table + 1
    Database["num_table"] = num_table
    return Database
Database = getDatabase(curDb)
print(Database)

def dec_to_bin(x, n):
    return format(x, f"0{n}b")
def generate_truthtable(data, list_of_qubitnum):

    df_bin = pd.DataFrame()
    for col, n in zip(data.columns, list_of_qubitnum):

        df_bin[col] = data[col].apply(dec_to_bin, args=(n,))

    df_expanded = pd.DataFrame()

    index = 0

    for col, n in zip(df_bin.columns, list_of_qubitnum):

        df_split = df_bin[col].astype(str).str.split("", expand=True)
        df_split = df_split.drop([0,n + 1], axis=1)
        for i in range(1,n + 1):
            if index < len(df_bin.columns) - 1:
                new_col = f"key{index}_{n - i}"
            else:
                new_col = f"value{0}_{n - i}"

            df_expanded[new_col] = df_split[i]

        index += 1

    return df_expanded

import itertools

def get_combinations(arr, n):
    result = []
    lengths = [len(sublist) for sublist in arr]
    total = sum(lengths)
    if n > total:
        return result
    if n == total:
        result.append({i: arr[i] for i in range(len(arr))})
        return result
    for indices in itertools.combinations(range(total), n):
        dic = {}
        count = 0
        for i, sublist in enumerate(arr):
            lst = []
            for j, element in enumerate(sublist):
                if count in indices:
                    lst.append(element)
        count += 1
        dic[i] = lst
        result.append(dic)

    return result

import copy
def delete_zeros(all_keys, zero_bits):
    result = []
    length_zero_bits = 0
    for i in range(len(zero_bits)):
        length_zero_bits += len(zero_bits[i])
    for n in range(length_zero_bits + 1):
        tobe_deleteds = get_combinations(zero_bits, n)

        for tobe_deleted in tobe_deleteds:

            temp_list = copy.deepcopy(all_keys)

            for k, v in tobe_deleted.items():
                for loc in v:

                    temp_list[k].remove(loc)
            result.append(temp_list)
    return result

def get_keyid_bitloc(key):
    s = key.replace("key", "")
    return s.split("_")

def get_valueid_bitloc(key):
    s = key.replace("value", "")
    return s.split("_")


def generate_Cnot_circuit(truth_table):

    truth_table_key = truth_table.filter(regex='^key')
    truth_table_value = truth_table.filter(regex='^value')

    result = {}

    key_cols = [col for col in truth_table_key.columns if col.startswith("key")]
    value_cols = [col for col in truth_table_value.columns if col.startswith("value")]

    for i, value_col in enumerate(value_cols):
        value_num, bit_num_value = get_valueid_bitloc(value_col)
        value_num = int(value_num)
        bit_num_value = int(bit_num_value)

        value_df = truth_table_value[[value_col]]

        merged_df = pd.concat([truth_table_key, value_df], axis=1)
        merged_df = merged_df.astype(int)
        merged_df = merged_df[merged_df[value_col] == 1]

        result[bit_num_value] = []

        for index, row in merged_df.iterrows():
            zero_bits = []
            for key_col in key_cols:
                key_num, bit_num = get_keyid_bitloc(key_col)
                key_num = int(key_num)
                bit_num = int(bit_num)
                if key_num >= len(zero_bits):
                    zero_bits.append([])
                if row[key_col] == 0:
                    zero_bits[key_num].append(bit_num)
            all_keys = []
            for key_col in key_cols:
                key_num, bit_num = get_keyid_bitloc(key_col)
                key_num = int(key_num)
                bit_num = int(bit_num)

                if key_num >= len(all_keys):
                    all_keys.append([])
                all_keys[key_num].append(bit_num)

            new_keys = delete_zeros(all_keys, zero_bits)
            for new_key in new_keys:
                result[bit_num_value].append(new_key)
        from collections import Counter
        counter = Counter(tuple(map(tuple, keys)) for keys in result[bit_num_value])
        result[bit_num_value] = [list(map(list, keys)) for keys, count in counter.items() if count % 2 == 1] # 只保留出现奇数次的二维列表

    return result

def apply_cnot_circuuit(dist_cnot_circuit, qc, names_key_register, name_value_register):
    qc_registers = qc.qregs

    reg_for_subcircuit_key = []
    for name in names_key_register:
        for reg in qc_registers:
            if reg.name == name:
                reg_for_subcircuit_key.append(reg)
                break

    reg_for_subcircuit_value = []
    for reg in qc_registers:

        if reg.name == name_value_register[0]:
            reg_for_subcircuit_value.append(reg)
            break
    reg_for_subcircuit = reg_for_subcircuit_key + reg_for_subcircuit_value

    sub_circuit = QuantumCircuit(*reg_for_subcircuit)

    lst_starting_point = []
    point = 0
    for reg in reg_for_subcircuit:
        lst_starting_point.append(point)
        point = point + reg.size

    for k, v in dist_cnot_circuit.items():
        for lst_2d in v:
            control_bit = []
            index = 0

            for lst_1d in lst_2d:

                control_bit = control_bit + [x + lst_starting_point[index] for x in lst_1d]
                index += 1
            target_bit = []
            target_bit = [] + [k + lst_starting_point[-1]]

            if control_bit:
                sub_circuit.mct(control_bit, target_bit, None, mode="noancilla")
            else:
                sub_circuit.x(target_bit)

    return sub_circuit



def create_states(num_states, num_qubit):
    states_register = QuantumRegister(num_qubit, "states")
    ancilla_qubit = QuantumRegister(1, "ancilla_qubit")
    states_circuit = QuantumCircuit(states_register, ancilla_qubit)
    num_H = math.floor(math.log(num_states, 2))

    if num_H > 0:
        states_circuit.h(range(num_H))
    num_rest_states = int(num_states - math.pow(2, num_H))

    if num_rest_states == 0:
        return states_circuit
    
    
    
    data= pd.DataFrame({
        "key0":[i for i in range(num_rest_states)],
        "value0":[1 for i in range(num_rest_states)]
    })

    truth_table = generate_truthtable(data, [num_qubit, 1])


    dist_cnot_circuit = generate_Cnot_circuit(truth_table)
 
    
    sub_circuit = apply_cnot_circuuit(dist_cnot_circuit, states_circuit, ["states"], ["ancilla_qubit"])



    states_circuit.compose(sub_circuit, qubits = states_register[:] + ancilla_qubit[:], inplace=True)

    
    from qiskit.circuit.library.standard_gates import HGate
    ch_gate = HGate().control(1)

    
    states_circuit.append(ch_gate, ancilla_qubit[:] + [states_register[num_H]])
    
    states_circuit.compose(sub_circuit, qubits= states_register[:] + ancilla_qubit[:], inplace=True)

    data_new = pd.DataFrame({
        "key0" : [i for i in range(int(math.pow(2, num_H)), int(math.pow(2, num_H)) + num_rest_states)],
        "value0" : [1 for i in range(int(math.pow(2, num_H)), int(math.pow(2, num_H)) + num_rest_states)]
    })
    truth_table_new = generate_truthtable(data_new, [num_qubit, 1])
    dist_cnot_circuit_new = generate_Cnot_circuit(truth_table_new)
    sub_circuit_new = apply_cnot_circuuit(dist_cnot_circuit_new, states_circuit, ["states"], ["ancilla_qubit"])
    states_circuit.compose(sub_circuit_new, qubits = states_register[:] + ancilla_qubit[:], inplace=True)
    
    return states_circuit

def NumOfQubits(x):
    return max(math.ceil(math.log(x, 2)), 1)

def get_ctrl_state(ctrl_value, num_qubits):
    return str(format(ctrl_value, f"0{num_qubits}b"))

def preparation(Database):
    num_table = Database['num_table']
    
    max_num_column = -1
    max_num_id = -1
    for table in Database['tables']:
        if table['num_attributes'] > max_num_column:
            max_num_column = table['num_attributes']
        if table['num_row'] > max_num_id:
            max_num_id = table['num_row']
    num_qubit_column = NumOfQubits(max_num_column)
    num_qubit_tableid = NumOfQubits(Database['num_table'])
    num_qubit_data = Num_Qubit_Data 
    num_qubit_id = NumOfQubits(max_num_id) 
    num_qubit_ancilla = 1
    qr_column = QuantumRegister(num_qubit_column, "column")
    qr_tableid = QuantumRegister(num_qubit_tableid, "tableid")
    qr_data = QuantumRegister(num_qubit_data, "data")
    qr_id = QuantumRegister(num_qubit_id, 'id')
    qr_ancilla = QuantumRegister(1, "ancilla")
    cr = ClassicalRegister(num_qubit_tableid + num_qubit_column + num_qubit_id + num_qubit_data)
    qc = QuantumCircuit(qr_tableid, qr_column, qr_id, qr_data, qr_ancilla, cr)

    sub_circuit_tableid = create_states(num_table, num_qubit_tableid)
    
    
    qc.compose(sub_circuit_tableid, qr_tableid[:] + qr_ancilla[:], inplace = True)

    for table in Database["tables"]:
        table_id_temp = table["table_id"]
        num_attributes_temp = table["num_attributes"]

        num_row_temp = table["num_row"]


        sub_circuit_attributes = create_states(num_attributes_temp, num_qubit_column)
        sub_circuit_attributes.name = "Prepare column id register"

        ctrl_state = get_ctrl_state(table_id_temp, num_qubit_tableid)

        ctrl_sub_circuit_attributes =  sub_circuit_attributes.control(num_qubit_tableid, ctrl_state = ctrl_state)
        
        
        qc.compose(ctrl_sub_circuit_attributes, qr_tableid[:] + qr_column[:] + qr_ancilla[:], inplace= True)

        sub_circuit_row = create_states(num_row_temp, num_qubit_id)
        sub_circuit_row.name = "Prepare row id register"

        ctrl_state = get_ctrl_state(table_id_temp, num_qubit_tableid)


        ctrl_sub_circuit_row = sub_circuit_row.control(num_qubit_tableid, ctrl_state = ctrl_state)
        qc.compose(ctrl_sub_circuit_row, qr_tableid[:] + qr_id[:] + qr_ancilla[:], inplace=True)
        

        
    return qc


def quantum_storage(Database):
    qc = preparation(Database)
    for table in Database["tables"]:
        relation_temp = table["relation"]
        num_attributes_temp = table["num_attributes"]
        num_row_temp = table["num_row"]
        isPrint = False
        for i in range(num_attributes_temp):

            df_temp = relation_temp[['tableid', "id", str(i)]].copy()
            df_temp.insert(loc=1, column='columnid', value=[i for x in range(num_row_temp)])

            list_of_qubitnum = []
            Qregs = {}
            for reg in qc.qregs:
                if reg.name != "ancilla":
                    list_of_qubitnum.append(reg.size)
                    Qregs[reg.name] = reg
            truth_table_temp = generate_truthtable(df_temp, list_of_qubitnum)
            dist_cnot_circuit_temp = generate_Cnot_circuit(truth_table_temp)
            sub_circuit = apply_cnot_circuuit(dist_cnot_circuit_temp, qc, names_key_register= ["tableid", "column", "id"],name_value_register=["data"])
            if isPrint == False:
                isPrint = True
            qc.compose(sub_circuit, Qregs["tableid"][:] + Qregs["column"][:] + Qregs["id"][:] + Qregs["data"][:], inplace = True)
    
    return qc
qc = quantum_storage(Database)
print("****************************************")
print("         QCOS has been implemented!")
print("****************************************")
qc.measure(range(qc.num_clbits), range(qc.num_clbits))
qc.draw()

{'tables': [{'table_id': 0, 'num_attributes': 2, 'num_row': 2, 'relation':        tableid  id  0  1
row_0        0   0  0  1
row_1        0   1  1  0}], 'num_table': 1}
****************************************
         QCOS has been implemented!
****************************************


/var/folders/f4/6bt5s2514m74026zjyvfxn7r0000gn/T/ipykernel_99456/4224436604.py:48: DeprecationWarning: 'U' mode is deprecated
  file = open(os.path.join(pathName, i), "rU")


»
tableid: ───────────────o────────────────────────────o────────────────■───────»
         ┌──────────────┴──────────────┐             │                │       »
 column: ┤0                            ├─────────────┼────────────────■───────»
         │                             │┌────────────┴─────────────┐  │       »
     id: ┤                             ├┤0                         ├──■────■──»
         │  Prepare column id register ││                          │┌─┴─┐┌─┴─┐»
   data: ┤                             ├┤  Prepare row id register ├┤ X ├┤ X ├»
         │                             ││                          │└───┘└───┘»
ancilla: ┤1                            ├┤1                         ├──────────»
         └─────────────────────────────┘└──────────────────────────┘          »
   c0: 4/═════════════════════════════════════════════════════════════════════»
                                                                              »
«                      ┌─┐      
«tableid: ──■──────────┤M├──────
«           │          └╥┘┌─┐   
« column: ──■───────■───╫─┤M├───
«           │  ┌─┐  │   ║ └╥┘   
«     id: ──┼──┤M├──┼───╫──╫────
«         ┌─┴─┐└╥┘┌─┴─┐ ║  ║ ┌─┐
«   data: ┤ X ├─╫─┤ X ├─╫──╫─┤M├
«         └───┘ ║ └───┘ ║  ║ └╥┘
«ancilla: ──────╫───────╫──╫──╫─
«               ║       ║  ║  ║ 
«   c0: 4/══════╩═══════╩══╩══╩═
«               2       0  1  3

In [2]:
from qiskit import QuantumCircuit, transpile, assemble
from qiskit import IBMQ, Aer
from qiskit.providers.ibmq import least_busy
from qiskit.tools.jupyter import *
from qiskit.visualization import *
from qiskit.tools.monitor import job_monitor

import json

backend = Aer.get_backend('qasm_simulator')

transpiled_circuit = transpile(qc, backend, optimization_level=0)
gate_counts = qc.count_ops()

dataQuantity = 0
for table in Database['tables']:
    dataQuantity += table["num_row"] * table["num_attributes"]
mct_counts = gate_counts.get('mcx', 0) + gate_counts.get('x', 0) + gate_counts.get('cx', 0) + gate_counts.get('ccx', 0)

circuitInfo = {}
circuitInfo['Database'] = '/dataset_3_0'
circuitInfo['dataQuantity'] = dataQuantity
circuitInfo['Qubits'] = transpiled_circuit.num_qubits
circuitInfo['Depth'] = transpiled_circuit.depth()
circuitInfo['Gates'] = qc.size()
circuitInfo['MCTGates'] = mct_counts
circuitInfo["Backend"] = str(backend)

with open("circuitInfo_column_0.json", "w") as outfile:
    json.dump(circuitInfo, outfile)

job = backend.run(transpiled_circuit)

result = job.result()
counts = result.get_counts(qc)

counts = result.get_counts(qc)

counts_json = json.dumps(counts)

with open('counts_column_0.json', 'w') as f:
    f.write(counts_json)

job_monitor(job)

print("Finished!")

/Users/litadou/opt/anaconda3/lib/python3.9/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.25.2
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


NameError: name 'qc' is not defined

In [37]:
result_dict = circuitInfo

def toStrQCOS(Database):
    
    num_table = Database['num_table']
    
    max_num_column = -1
    max_num_id = -1
    for table in Database['tables']:
        if table['num_attributes'] > max_num_column:
            max_num_column = table['num_attributes']
        if table['num_row'] > max_num_id:
            max_num_id = table['num_row']
    num_qubit_column = NumOfQubits(max_num_column)
    num_qubit_tableid = NumOfQubits(Database['num_table'])
    num_qubit_data = Num_Qubit_Data 
    num_qubit_id = NumOfQubits(max_num_id) 
    
    tables = Database["tables"]

    resultStrQCOS = []
    
    for table in tables:
        relation_temp = table["relation"]

        for index, row in relation_temp.iterrows():
            strQCOSPrefix = ""
            strQCOSPrefix = dec_to_bin(row.iloc[0], num_qubit_tableid) + strQCOSPrefix

            columnID = 0
            for col in row.iloc[2:]:

                strQCOS = strQCOSPrefix

                bin_temp_columnID = dec_to_bin(columnID, num_qubit_column)
                strQCOS = bin_temp_columnID + strQCOS
                columnID += 1
                strQCOS = dec_to_bin(row.iloc[1], num_qubit_id) + strQCOS

                bin_temp_data = dec_to_bin(col, num_qubit_data)
                strQCOS = bin_temp_data + strQCOS

                resultStrQCOS.append(strQCOS)

    
    return resultStrQCOS
        
resultStrQCOS = toStrQCOS(Database)
print(resultStrQCOS)  

    
def calculate_ratio(measurement_results, correct_results):
    total_correct_count = 0
    total_measurement_count = sum(measurement_results.values())

    for result in correct_results:
        total_correct_count += measurement_results.get(result, 0)

    ratio = total_correct_count / total_measurement_count
    return ratio



def QCOSRatio(Database):
    databaseStr = toStrQCOS(Database)
    Ratio=calculate_ratio(counts, databaseStr)
    return Ratio

print(counts)
print(QCOSRatio(Database))
print("Dnoe!")

def calculate_integrity(lst, dct):
    count = sum([1 for item in lst if item in dct])
    percentage = count / len(lst)
    return percentage

integrityRatio = calculate_integrity(resultStrQCOS, counts)
print(integrityRatio)


result_dict["Valid Ratio"] = QCOSRatio(Database)
result_dict["numDataQubits"] = Num_Qubit_Data
result_dict["integrityRatio"] = calculate_integrity(resultStrQCOS, counts)
print(result_dict)

# with open('circuitInfo_column.json', 'w') as f:
#     json.dump(result_dict, f)

['0000', '1010', '1100', '0110']
{'1100': 237, '1010': 251, '0110': 273, '0000': 263}
1.0
Dnoe!
1.0
{'Database': '/dataset_3_0', 'dataQuantity': 4, 'Qubits': 5, 'Depth': 11, 'Gates': 10, 'MCTGates': 4, 'Backend': 'qasm_simulator', 'Valid Ratio': 1.0, 'numDataQubits': 1, 'integrityRatio': 1.0}
